## 基於視窗滑動策略的機器閱讀理解(MRC)
### 載人套件

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForQuestionAnswering,TrainingArguments,Trainer, DefaultDataCollator

### 下載資料集

In [ ]:
datasets = load_dataset("roberthsu2003/for_MRC_QA", cache_dir='data')
datasets

In [4]:
datasets["train"][0]

{'id': '1001-10-1',
 'context': '2010年引進的廣州快速公交運輸系統，屬世界第二大快速公交系統，日常載客量可達100萬人次，高峰時期每小時單向客流高達26900人次，僅次於波哥大的快速交通系統，平均每10秒鐘就有一輛巴士，每輛巴士單向行駛350小時。包括橋樑在內的站台是世界最長的州快速公交運輸系統站台，長達260米。目前廣州市區的計程車和公共汽車主要使用液化石油氣作燃料，部分公共汽車更使用油電、氣電混合動力技術。2012年底開始投放液化天然氣燃料的公共汽車，2014年6月開始投放液化天然氣插電式混合動力公共汽車，以取代液化石油氣公共汽車。2007年1月16日，廣州市政府全面禁止在市區內駕駛摩托車。違反禁令的機動車將會予以沒收。廣州市交通局聲稱禁令的施行，使得交通擁擠問題和車禍大幅減少。廣州白雲國際機場位於白雲區與花都區交界，2004年8月5日正式投入運營，屬中國交通情況第二繁忙的機場。該機場取代了原先位於市中心的無法滿足日益增長航空需求的舊機場。目前機場有三條飛機跑道，成為國內第三個擁有三跑道的民航機場。比鄰近的香港國際機場第三跑道預計的2023年落成早8年。',
 'question': '廣州的快速公交運輸系統每多久就會有一輛巴士？',
 'answers': {'answer_start': [84], 'text': ['10秒鐘']}}

### 數據處理

In [5]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

In [ ]:
##選擇前10筆資料處理(讓邏輯簡化)
sample_dataset = datasets['train'].select(range(10))
sample_dataset[:]

In [16]:
#前面放Question,後面放context
#增加requrn_overflowing_tokens
#max_length先設為384,比較好明
from pprint import pprint
tokenized_example = tokenizer(
    text = sample_dataset["question"],
    text_pair=sample_dataset['context'],
    return_offsets_mapping=True,
    return_overflowing_tokens=True,
    max_length=384,
    truncation="only_second",
    padding="max_length"
)
#pprint(tokenized_example,compact=True)
print(tokenized_example.keys()) #多了offset_mapping,overflow_to_sample_mapping

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])


In [ ]:
#查看overflow_to_sample_mapping內容(切割資訊)
#一條數據因為長度超過,被切割為多條
tokenized_example['overflow_to_sample_mapping'], len(tokenized_example['overflow_to_sample_mapping']) #本來只有10條變為14條

([0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 9], 14)

In [30]:
#解碼前3條input_ids的資料,看切割後的狀態
pprint(tokenizer.batch_decode(tokenized_example['input_ids'][:3]),compact=True,width=1000) #改變width查看資料,中文被decode會有1格空格

['[CLS] 廣 州 的 快 速 公 交 運 輸 系 統 每 多 久 就 會 有 一 輛 巴 士 ？ [SEP] 2010 年 引 進 的 廣 州 快 速 公 交 運 輸 系 統 ， 屬 世 界 第 二 大 快 速 公 交 系 統 ， 日 常 載 客 量 可 達 100 萬 人 次 ， 高 峰 時 期 每 小 時 單 向 客 流 高 達 26900 人 次 ， 僅 次 於 波 哥 大 的 快 速 交 通 系 統 ， 平 均 每 10 秒 鐘 就 有 一 輛 巴 士 ， 每 輛 巴 士 單 向 行 駛 350 小 時 。 包 括 橋 樑 在 內 的 站 台 是 世 界 最 長 的 州 快 速 公 交 運 輸 系 統 站 台 ， 長 達 260 米 。 目 前 廣 州 市 區 的 計 程 車 和 公 共 汽 車 主 要 使 用 液 化 石 油 氣 作 燃 料 ， 部 分 公 共 汽 車 更 使 用 油 電 、 氣 電 混 合 動 力 技 術 。 2012 年 底 開 始 投 放 液 化 天 然 氣 燃 料 的 公 共 汽 車 ， 2014 年 6 月 開 始 投 放 液 化 天 然 氣 插 電 式 混 合 動 力 公 共 汽 車 ， 以 取 代 液 化 石 油 氣 公 共 汽 車 。 2007 年 1 月 16 日 ， 廣 州 市 政 府 全 面 禁 止 在 市 區 內 駕 駛 摩 托 車 。 違 反 禁 令 的 機 動 車 將 會 予 以 沒 收 。 廣 州 市 交 通 局 聲 稱 禁 令 的 施 行 ， 使 得 交 通 擁 擠 問 題 和 車 禍 大 幅 減 少 。 廣 州 白 雲 國 際 機 場 位 於 白 雲 區 與 花 都 區 交 界 ， 2004 年 8 月 5 日 正 式 投 入 運 營 ， 屬 中 國 交 通 情 況 第 二 繁 忙 的 機 場 。 該 機 場 [SEP]',
 '[CLS] 廣 州 的 快 速 公 交 運 輸 系 統 每 多 久 就 會 有 一 輛 巴 士 ？ [SEP] 取 代 了 原 先 位 於 市 中 心 的 無 法 滿 足 日 益 增 長 航 空 需 求 的 舊 機 場 。 目 前 機 場 有 三 條 飛 機 跑 道 ， 成 為 國 內 第 三 個 擁 有 三 跑 道 的 民 航 機 場 。 比 鄰 近 的 香 港 國 際 機